In [4]:
import ray
import torch
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator
from ray import train
from ray.air import session
from torch import nn

from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit import QuantumCircuit
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector

from quantum_serverless.train.trainer import (
    QiskitScalingConfig,
    QiskitTorchTrainer,
    get_runtime_session,
)

In [5]:
import numpy as np
import pandas as pd

data = pd.read_csv('./data/mnist-train.csv')
data = np.array(data)
data_binary = data[np.where(data[:,0]<2)[0]]

rows, cols = data_binary.shape
np.random.shuffle(data_binary)

#separate train data
train_data = data_binary[1000:2000].T
train_data[1:] = train_data[1:]/255. # normalizing pixels
train_data = train_data.T
print(train_data.shape)

#separate train data
dev_data = data_binary[:1000].T
dev_data[1:] = dev_data[1:]/255. # normalizing pixels
dev_labels = dev_data[0]
dev_data = dev_data[1:].T.reshape(-1, 1, 28, 28).astype(np.float32)
print(dev_data.shape)

(1000, 785)
(1000, 1, 28, 28)


In [8]:
import ray
train_ds = ray.data.from_numpy(train_data)
dev_ds = ray.data.from_numpy(dev_data)

In [9]:
INPUT_SIZE = 1
LAYER_SIZE = 2
OUTPUT_SIZE = 1
NUM_EPOCHS = 30

In [10]:
def create_qnn(session, layer_size):
    feature_map = ZZFeatureMap(layer_size)
    ansatz = RealAmplitudes(layer_size, reps=1)

    qc = QuantumCircuit(layer_size)
    qc.append(feature_map, range(layer_size))
    qc.append(ansatz, range(layer_size))

    qnn = EstimatorQNN(
        estimator=Estimator(session=session),
        circuit=qc,
        input_params=feature_map.parameters,
        weight_params=ansatz.parameters,
        input_gradients=True
    )
    return TorchConnector(qnn)

In [11]:
class NeuralNetwork(nn.Module):
    """Test neural network."""

    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(INPUT_SIZE, LAYER_SIZE)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(LAYER_SIZE, OUTPUT_SIZE)

    def forward(self, input_tensor):
        """Forward pass."""
        return self.layer2(self.relu(self.layer1(input_tensor)))

In [12]:
class HybridQNN(nn.Module):
    """Test neural network."""

    def __init__(self, session):
        super().__init__()
        self.layer1 = nn.Linear(INPUT_SIZE, LAYER_SIZE)
        self.relu = nn.ReLU()
        self.qnn = create_qnn(session, LAYER_SIZE)

    def forward(self, input_tensor):
        """Forward pass."""
        x = self.relu(self.layer1(input_tensor))
        x = self.qnn(x)
        return x

In [18]:
def train_loop(config):
    """Test training loop."""
    runtime_session = get_runtime_session(config)
    print("Session", runtime_session)

    is_qnn = config.get("is_qnn")
    dataset_shard = session.get_dataset_shard("train")
    loss_fn = nn.MSELoss()

    print("DS Shard: ", dataset_shard)

    if is_qnn:
        model = HybridQNN(runtime_session)
    else:
        model = NeuralNetwork()

    print("Model: ", model)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

    model = train.torch.prepare_model(model)

    for epoch in range(NUM_EPOCHS):
        for batch in dataset_shard.iter_torch_batches(batch_size=32, dtypes=torch.float):
            labels = torch.from_numpy(batch[:,0]).long()
            inputs = torch.from_numpy(batch[:,1:].reshape(-1, 1, 28, 28)).float()
            # inputs, labels = torch.unsqueeze(batches["x"], 1), batches["y"]
            output = model(inputs)
            loss = loss_fn(output, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            print(f"epoch: {epoch}, loss: {loss.item()}")

In [19]:
from tokens import Tokens
API_TOKEN = Tokens.API_TOKEN[0]
QiskitRuntimeService.save_account(channel="ibm_quantum",
                                  token=API_TOKEN,
                                  overwrite=True)

In [20]:
"""Tests trainer."""
train_dataset = ray.data.from_items(
    [{"x": x, "y": 2 * x + 1} for x in range(200)]
)

In [21]:
scaling_config = QiskitScalingConfig(num_workers=2, num_qubits=2, simulator=True)

In [22]:
runtime_service = QiskitRuntimeService(channel="ibm_quantum")
print("runtime service: ", runtime_service)

runtime service:  <QiskitRuntimeService>


In [23]:
trainer = QiskitTorchTrainer(
    train_loop_per_worker=train_loop,
    qiskit_runtime_service_account=runtime_service.active_account(),
    scaling_config=scaling_config,
    datasets={"train": train_ds},
    train_loop_config={"is_qnn": True},
)

In [ ]:
trainer.fit()

(RayTrainWorker pid=77522) runtime_job._start_websocket_client:WARNING:2023-01-26 16:12:42,976: An error occurred while streaming results from the server for job cf99gkepkn0k7a2c4os0:
(RayTrainWorker pid=77522) Traceback (most recent call last):
(RayTrainWorker pid=77522)   File "/opt/anaconda3/envs/ray_train/lib/python3.9/site-packages/qiskit_ibm_runtime/runtime_job.py", line 454, in _start_websocket_client
(RayTrainWorker pid=77522)     self._ws_client.job_results()
(RayTrainWorker pid=77522)   File "/opt/anaconda3/envs/ray_train/lib/python3.9/site-packages/qiskit_ibm_runtime/api/clients/runtime_ws.py", line 70, in job_results
(RayTrainWorker pid=77522)     self.stream(url=url, retries=max_retries, backoff_factor=backoff_factor)
(RayTrainWorker pid=77522)   File "/opt/anaconda3/envs/ray_train/lib/python3.9/site-packages/qiskit_ibm_runtime/api/clients/base.py", line 230, in stream
(RayTrainWorker pid=77522)     raise WebsocketError(error_message)
(RayTrainWorker pid=77522) qiskit_ibm_